In [0]:
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [0]:
import econsir as es
import numpy as np
import pandas as pd

### Simulate

In [0]:
T0 = 300
K0 = 1000

In [0]:
# make random start times
start = np.random.randint(10, 30, size=K0)
imp0 = es.one_hot(start, value=1/1e6, T=T0)

In [0]:
# set up parameters
par0 = es.load_args('params/default.toml')
par0['β'] = np.random.lognormal(np.log(par0['β']), 0.1, size=K0)
par0['ψ'] = np.random.lognormal(np.log(par0['ψ']), 0.1, size=K0)

In [0]:
pol0 = {'zb': 0.8, 'zc': 30.0}
sim0, _ = es.simulate_path(par0, pol0, imp=imp0, K=K0, T=T0)

In [0]:
ch = es.outcome_summary(sim0)
ch.save('output/simul.svg')
ch

### Load Data

In [0]:
data = es.load_data('data/panel_usa.csv', min_date='2020-02-15', min_pop=50_000)
data_df = es.framify(data, data['date'], data['fips'])

In [0]:
T, K = data['T'], data['K']
wgt, fips = data['wgt'], data['fips']
dates = data['date']
print(T, K)

In [0]:
es.outcome_summary(data_df, ao_lim=120, wgt=wgt)

### Estimate

In [0]:
# set up parameters
par = es.load_args('params/default.toml')
par['β'] = np.ones(K)*par['β']
par['ψ'] = np.ones(K)*par['ψ']
par['ϝ'] = 20.0

In [0]:
hard = {'ρ': 0.15, 'ψ0': 1.0, 'ψr': 0.0}
par_est = es.estimate(par, data, hard_params=hard, R=1000, per=250)

In [0]:
es.save_args(par_est, 'params/estim.toml')

In [0]:
pol_est = {'zb': es.calc_zbar(par, data), 'zc': 0, 'kz': 0, 'vx': 0}
sim_est, st1_est = es.simulate_path(par_est, pol=pol_est, imp=data['imp'], locs=data['fips'], T=T, K=K)
es.outcome_summary(sim_est, wgt=wgt)

### Optimal Policy

In [0]:
T1 = 365
date_opt = dates[-1]

In [0]:
hard_pol = {'kz': 100.0, 'vx': 0.0}
pol_init = {'zb': 1.0, 'zc': 20.0}
pol_opt = es.optimal_policy(
    pol_init, par_est, st0=st1_est, wgt=wgt, T=T1, K=K,
    hard_policy=hard_pol, long_run=True,
)

In [0]:
print(es.eval_policy(hard_pol, par_est, st0=st1_est, wgt=wgt, T=4*T1, K=K))
print(es.eval_policy(pol_opt, par_est, st0=st1_est, wgt=wgt, T=T1, K=K))

In [0]:
sim_opt, _ = es.simulate_path(par_est, pol_opt, st0=st1_est, locs=fips, T=120, K=K, date=date_opt)
es.outcome_summary(sim_opt, wgt=wgt)